In [202]:
%ls

homework_modules.ipynb  sample_data/


In [203]:
%run homework_modules.ipynb


In [204]:
%matplotlib inline
!pip install mnist
!pip install scikit-learn
from time import time, sleep
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython import display
import sklearn

In [205]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [206]:
import mnist
images = mnist.train_images() / 255
labels = mnist.train_labels()
pca = PCA(n_components=2)
new_features = pca.fit_transform([i.reshape(-1) for i in images])
X_train, X_val, y_train, y_val = train_test_split(images, labels)

In [207]:
def simple_sgd(variables, gradients, config, state):
    # 'variables' and 'gradients' have complex structure, accumulated_grads will be stored in a simpler one
    state.setdefault('accumulated_grads', {})

    var_index = 0
    for current_layer_vars, current_layer_grads in zip(variables, gradients):
        for current_var, current_grad in zip(current_layer_vars, current_layer_grads):

            current_var -= config['learning_rate'] * current_grad
            var_index += 1

In [208]:
net = Sequential()
net.add(Linear(28*28, 10))
net.add(LogSoftMax())

criterion = ClassNLLCriterion()

print(net)

Linear 784 -> 10
LogSoftMax



In [209]:
# Optimizer params
optimizer_config = {'learning_rate' : 1e-2}
optimizer_state = {}

# Looping params
n_epoch = 20
batch_size = 512

In [210]:
# batch generator
def get_batches(dataset, batch_size):
    X, Y = dataset
    n_samples = X.shape[0]

    # Shuffle at the start of epoch
    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)

        batch_idx = indices[start:end]

        yield X[batch_idx], Y[batch_idx]

In [211]:
def train():
    train_loss_history = []
    val_loss_history = [0,]
    val_acc_history = [0,]
    steps = [0,]

    for i in range(n_epoch):
        steps.append(steps[-1])
        for x_batch, y_batch in get_batches((X_train, y_train), batch_size):

            net.zeroGradParameters()
            x_batch = x_batch.reshape(-1, 28*28)

            # Forward
            predictions = net.forward(x_batch)
            loss = criterion.forward(predictions, y_batch)

            # Backward
            dp = criterion.backward(predictions, y_batch)
            net.backward(x_batch, dp)

            # Update weights
            simple_sgd(net.getParameters(),
                    net.getGradParameters(),
                    optimizer_config,
                    optimizer_state)

            train_loss_history.append(loss)
            steps[-1] += 1

        sum_loss = 0
        sum_acc = 0
        count_val_steps = 0
        for x_batch, y_batch in get_batches((X_val, y_val), batch_size):
            x_batch = x_batch.reshape(-1, 28*28)
            predictions = net.forward(x_batch)
            loss = criterion.forward(predictions, y_batch)
            sum_loss += loss
            sum_acc += accuracy_score(y_batch, np.argmax(predictions, axis=1))
            count_val_steps += 1

        val_loss_history.append(sum_loss / count_val_steps)
        val_acc_history.append(sum_acc / count_val_steps)

        # Visualize
        display.clear_output(wait=True)
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        ax[0].set_title("Training loss")
        ax[0].set_xlabel("#iteration")
        ax[0].set_ylabel("loss")
        ax[0].plot(train_loss_history, 'b')
        ax[0].grid()

        ax[1].set_title("Validation loss")
        ax[1].set_xlabel("#iteration")
        ax[1].set_ylabel("loss")
        ax[1].plot(steps, val_loss_history, 'b')
        ax[1].grid()

        ax[2].set_title("Validation Accuracy")
        ax[2].set_xlabel("#iteration")
        ax[2].set_ylabel("accuracy")
        ax[2].plot(steps, val_acc_history, 'b')
        ax[2].grid()

        plt.show()

        print('Current loss: %f' % (sum_loss / count_val_steps))



In [212]:
train()

KeyboardInterrupt: 

In [ ]:
i = 10
k = 100
X_test = X_test.reshape([10000, 28,28])
plt.imshow(X_test[i], cmap='binary')
X_test = X_test.reshape([-1, 28*28])

plt.axis('off')
print(y_train[range(k)])
#print(X_test[i].shape)
X = X_test[range(k)]
print(net2.forward(X))
